In [2]:
import numpy as np
import cv2 
from matplotlib import pyplot as plt
import os
import glob

In [93]:
def ssd(vecs1, vecs2, window_size):
    # print(vecs1.shape , vecs1)
    # print(vecs2.shape, vecs2)
    vecs1 = vecs1.flatten().reshape((2*window_size+1)**2, 3).astype(int)
    # print(vecs1)
    vecs2 = vecs2.flatten().reshape((2*window_size+1)**2, 3).astype(int)
    # print(vecs2)
    
    output = np.sum([np.sum((x)**2) for x in (vecs1-vecs2)])

    return output

Looping
---

Testing approach where we loop over every pixel. Way too slow. Takes about 1 min to do the calculation for a single row of x values in the given image

In [ ]:

base_path  = os.getcwd() + "/Dataset/"
pathL = '2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg'
pathR = '2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg'

print(base_path + pathL)

window_size = 1 ## This is not the size of the actual window, our window looks like this [window_size :pixel of interest: window_size], so the size of the whole window is (2*window size)+1


left = cv2.imread(base_path + pathL)
right = cv2.imread(base_path + pathR)

print("Shape before padding ", left.shape)
print("Shape before padding ", right.shape)

left = cv2.copyMakeBorder(left, window_size, window_size, window_size, window_size, cv2.BORDER_CONSTANT, None, value = 0)
right = cv2.copyMakeBorder(right, window_size, window_size, window_size, window_size, cv2.BORDER_CONSTANT, None, value = 0)


# plt.imshow(left)
# plt.axis('off')
# plt.show()
# plt.imshow(right)
# plt.axis('off')
# plt.show()


print("Shape after padding ", left.shape)
print("Shape after padding ", right.shape)

x = left.shape[1] 
y = left.shape[0]


disparity = []

for i_y in range(1,2):
# for i_y in range(window_size, y-window_size):

    print(i_y-window_size)


    right_line_nbhds = []

    print("Before right x nbhds created")
    ## Create all the pixel neighbourhoods for the line in the right image
    for i_x2 in range(window_size, x-window_size):
            
        nbhd_right_img = np.array(right[i_y-window_size:i_y+window_size+1, i_x2-window_size:i_x2+window_size+1,:]).astype(int) 
        right_line_nbhds.append(nbhd_right_img)

    print("After right x nbhds created")

    right_line_nbhds = np.array(right_line_nbhds)

    print("nbhds ",len(right_line_nbhds))

    ## For each x in the left image, compare it to every pixel neighbourhood in the right image (using SSD here) and take the smallest SSD
    for i_x in range(window_size, x-window_size):
        
        x_left = left[i_y-window_size:i_y + window_size + 1,i_x - window_size:i_x+window_size+1,:] ## Left pixel neighbourhood
        nbhd_ssd = np.array([ssd(x_left,r, window_size) for r in right_line_nbhds]) ## Generate all SSDs for left pixel (1 per right pixel nbhd) 
        match_r_img = np.argmin(nbhd_ssd) ## Take minimum ssd as the closest match       
        pixel_disparity = (i_x - window_size) - match_r_img ## Calculate disparity

        # print(i_x, match_r_img,pixel_disparity, nbhd_ssd )
        # print(i_x, match_r_img)

        disparity.append(pixel_disparity)

    
print(disparity)

        




Vectorized Version 1
--- 

This brings the time down to about 2.5 seconds per row. Still not fast enough.

The way this method works is that for a given row and all the neighbourhoods related to that row it turns them into a vector and performs vector operations to calculate our disparity. 

The next step is to vectorize the whole image and perform these operations

In [109]:

base_path  = os.getcwd() + "/Dataset/"
pathL = '2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg'
pathR = '2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg'

print(base_path + pathL)

window_size = 1 ## This is not the size of the actual window, our window looks like this [window_size :pixel of interest: window_size], so the size of the whole window is (2*window size)+1


left = cv2.imread(base_path + pathL)
right = cv2.imread(base_path + pathR)

print("Shape before padding ", left.shape)
print("Shape before padding ", right.shape)

left = cv2.copyMakeBorder(left, window_size, window_size, window_size, window_size, cv2.BORDER_CONSTANT, None, value = 0)
right = cv2.copyMakeBorder(right, window_size, window_size, window_size, window_size, cv2.BORDER_CONSTANT, None, value = 0)



print("Shape after padding ", left.shape)
print("Shape after padding ", right.shape)

x = left.shape[1] 
y = left.shape[0]

# print(window_size, x-window_size)

disparity = []

for i_y in range(1,10):
# for i_y in range(window_size, y-window_size):
    print(i_y-window_size)



    right_line_nbhds = []
    # print("Before right x nbhds created")

    for i_x2 in range(window_size, x-window_size):

    # for i_x2 in range (1,3):

        nbhd_right_img = np.array(right[i_y-window_size:i_y+window_size+1, i_x2-window_size:i_x2+window_size+1,:]).astype(int).flatten() ##.reshape(1,window_size**2)
        # print("right nbhd ", nbhd_right_img)
        right_line_nbhds.append(nbhd_right_img)

    # print("After right x nbhds created")
    
    # print("right ", right_line_nbhds[0:5])

    right_line_nbhds = np.array(right_line_nbhds).flatten()

    right_line_nbhds = np.tile(right_line_nbhds, x-2*window_size)


    left_line_nbhds = []

    # print("Before left x nbhds created")


    for i_x2 in range (window_size, x - window_size):

    # for i_x2 in range (1,3):

        nbhd_left_img = np.array(left[i_y-window_size:i_y+window_size+1, i_x2-window_size:i_x2+window_size+1,:]).astype(int).flatten()
        
        # print("left nbhd ", nbhd_left_img)

        nbhd_left_img = np.tile(nbhd_left_img, x - 2*window_size)

        left_line_nbhds.append(nbhd_left_img)


    left_line_nbhds = np.array(left_line_nbhds).flatten()

    # print("left " ,left_line_nbhds[0:5])

    # left_line_nbhds = np.array(left_line_nbhds).flatten()

    # left_line_nbhds = np.repeat(left_line_nbhds, x)
    # right_line_nbhds = np.tile(right_line_nbhds, (x,1))


    # print("After left x nbhds created")

    # print("right ", right_line_nbhds[0:5])
    # print("left " ,left_line_nbhds)

    # print("right ", right_line_nbhds.shape)
    # print("left " ,left_line_nbhds.shape)

    # print("sub ",(left_line_nbhds-right_line_nbhds)[0:54])

    # print("sub2 ",((left_line_nbhds-right_line_nbhds)**2)[0:54])

    # # print("sub2 ",((left_line_nbhds-right_line_nbhds)**2).flatten()[0:27])

    flat = ((left_line_nbhds-right_line_nbhds)**2)

    # print(flat.shape)

    # print(flat[0:27])
    # print(flat[27:54])
    # print(flat[54:81])

    # print("flat ", flat[0:27].sum())
    # print("flat ", flat[27:54].sum())
    # print("flat ", flat[54:81].sum())

    # print(len(flat)/27)

    step = ((2*window_size+1)**2)*3 ## Number of pixels in window * number of colour channels(3)
    waga = np.array([flat[i:i+step].sum() for i in range(0,len(flat), 27)])
    # print(waga.shape)
    # np.argmin(flat) 
    ssd = np.array([np.argmin(waga[i:i + (x - 2*window_size)]) for i in range (0, (x - 2*window_size)**2, (x - 2*window_size) )])

    # print(ssd.shape)

    # print(ssd)
    row_disparity = ssd - np.linspace(0, x - (2*window_size + 1) ,num = (x-2*window_size)).astype(int)
    disparity.append(row_disparity)
 


    

    # print(waga)

    # for l_x in left_line_nbhds:


    


    # print("nbhds ",len(right_line_nbhds))

    # # for i_x in range(10):
    # for i_x in range(window_size, x-window_size):
        
        

    #     x_left = left[i_y-window_size:i_y + window_size + 1,i_x - window_size:i_x+window_size+1,:]

    #     # nbhd_right_img = right[i_y-window_size:i_y+window_size+1, i_x-window_size:i_x+window_size+1,:] ## nbhd = neighbourhood

    #     nbhd_ssd = np.array([ssd(x_left,r, window_size) for r in right_line_nbhds]) ## Generate ssd for neighbourhood



    #     match_r_img = np.argmin(nbhd_ssd) ## Take minimum ssd as the closest match 
    #     pixel_disparity = (i_x - window_size) - match_r_img ## Calculate disparity
        
    #     print(i_x)

    #     disparity.append(pixel_disparity)


        
disparity




/Users/oliverbestel/Desktop/Documents/Work/COMP90086/Tuts/P1/CV_Project/Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg
Shape before padding  (400, 881, 3)
Shape before padding  (400, 881, 3)
Shape after padding  (402, 883, 3)
Shape after padding  (402, 883, 3)
0
1
2
3
4
5
6
7
8


[array([   0,   65,   65,   65,   -3,   -4,   -5,   -6,   -7,   -8,   -9,
          27,   26,   25,   24,   23,   22,   21,   20,  -17,  -17,  -16,
          -8,  -16,  -11,  -13,    9,  -23,    9,  -18,    6,    6,    6,
           5,  104,  103,  102,  101,    2,   29,   28,    0,    0,   -1,
          -1,   -1,   -1,   -1,   -1,   -1,   -2,   -1,   -2,   -2,   -2,
          -2,   -2,   -2,   -2,   -1,   -2,   -2,   -2,   -2,   -2,   -2,
          -2,   -2,   -3,   -2,   -3,   -3,  -71,  -72,  -73,  -74,  -75,
          -7,   60,   47,   46,   44,   43,    0,   -1,   24,  -51,  -52,
         -53,  -53,  -54,  -55,  -56,  -57,  -57,  -59,  -60,  -61,  -61,
         -87,  -88,  -88,  -89,  -90,  -91,  -92,  -92,  -93,  -94,  -73,
         -74,  -73,  -74,  -75,  -76,   11,   10,    9,    8,  -85,  -10,
         510,  510,  -10,  -10,  -10,  -10,  224,   -2,   -3,    6,    6,
          -7,   -7,  -48, -117, -100,  -98,  -99,  -99,   -2,   -1,   -2,
          -2,   -1,   -1,   -1,   -1, 

In [ ]:
def create_neighbourhood(pixel_x, pixel_y, img, window_size):

    nbhd = np.array(right[pixel_y-window_size:pixel_y+window_size+1, pixel_x-window_size:pixel_x+window_size+1,:])
    
    return nbhd

    

# def create_all_neighbourhoods(img, padding):


    